**Part 1: Harris Corner Detector**

In [1]:
# importing libraries

import cv2 
import sys

import numpy as np
import matplotlib as plt

np.set_printoptions(threshold=sys.maxsize)

In [2]:
# loading example images

imgs = {}

imgs['goi1'] = cv2.imread('CV_Lista02_dados/goi1.jpg', 0)
imgs['goi2'] = cv2.imread('CV_Lista02_dados/goi2.jpg', 0)
imgs['building1'] = cv2.imread('CV_Lista02_dados/building1.jpg', 0)
imgs['building2'] = cv2.imread('CV_Lista02_dados/building2.jpg', 0)

In [3]:
# Calculating derivatives for both directions
# for each image example

Wd = 3

dx = {}
dy = {}

for img in imgs.keys():
    dx[img] = cv2.Sobel(imgs[img], -1, 1, 0, ksize=Wd)
    dy[img] = cv2.Sobel(imgs[img], -1, 0, 1, ksize=Wd)

In [4]:
# Visualizing dx

for img in imgs.keys():
    cv2.imshow('img', dx[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
# Visualizing dy

for img in imgs.keys():
    cv2.imshow('img', dy[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
# Calculating the covariance matrix
# for each image example

corner = {}
width = {}
heigh = {}

offset = int(((5 * Wd) - 1) / 2)
threshold = 1500000
stride = 1

for img in imgs.keys():
    dxdx = dx[img] ** 2
    dydy = dy[img] ** 2
    dxdy = dx[img] * dy[img]
    
    # Initializing parameters
    width[img] = imgs[img].shape[0]
    heigh[img] = imgs[img].shape[1]

    # Initializing corner variable to store the points
    corner[img] = np.zeros((width[img], heigh[img]), np.float64)

    # Applying symmetric padding to the image
    dxdx_pad = np.pad(dxdx, ((offset, offset), (offset, offset)), 'constant')
    dydy_pad = np.pad(dydy, ((offset, offset), (offset, offset)), 'constant')
    dxdy_pad = np.pad(dxdy, ((offset, offset), (offset, offset)), 'constant')

    # For each pixel, the covariance matrix is calculated
    for y in range(heigh[img]):
        for x in range(width[img]):
            sdxdx = np.sum(dxdx_pad[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])
            sdydy = np.sum(dydy_pad[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])
            sdxdy = np.sum(dxdy_pad[(x * stride) : (x * stride) + (2 * offset), (y * stride) : (y * stride) + (2 * offset)])

            cov = [[sdxdx, sdxdy], [sdxdy, sdydy]]

            lamb1 = np.linalg.eig(cov)[0][0]
            lamb2 = np.linalg.eig(cov)[0][1]

            R = ((lamb1 * lamb2) - 0.04 * ((lamb1 + lamb2) ** 2))

            if R > threshold:
                corner[img][x, y] += R

In [7]:
# Visualizing the points detected as corners

for img in imgs.keys():
    cv2.imshow('img', corner[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
# Setting a window Wh to verify which corners are to be considered
# It works like a "maxpooling layer"

Wh = 20
idxs = []
filt_corner = {}

for img in imgs.keys():
    filt_corner[img] = np.zeros((width[img], heigh[img], 1), np.float64)

    for y in range (heigh[img] // Wh):
        for x in range(width[img] // Wh):
            idx = np.argmax(corner[img][(x * Wh) : ((x + 1) * Wh), (y * Wh) : ((y + 1) * Wh)])

            if (idx != 0):
                new_y = ((y * Wh) + (idx % Wh))
                new_x = ((x * Wh) + (idx // Wh))
                idxs.append(idx)
                filt_corner[img][new_x, new_y] = 255

In [9]:
# Visualizing

for img in imgs.keys():
    cv2.imshow('img', filt_corner[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
# Image process for better visualization

kernel = np.ones((Wd, Wd), np.uint8)
fc_dil = {}
visualization = {}

for img in imgs.keys():
    # Dilatating the dots
    fc_dil[img] = cv2.dilate(filt_corner[img], kernel, iterations=1)
    
    # Converting to uint8
    fc_dil[img] = fc_dil[img].astype(np.uint8)
    
    # Getting the negative
    fc_dil[img] = cv2.bitwise_not(fc_dil[img])
    
    # Appling the mask
    visualization[img] = cv2.bitwise_and(imgs[img], imgs[img], mask = fc_dil[img])
    
    # Visualizing the processed corners
    cv2.imshow('img', fc_dil[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # And the final result
    cv2.imshow('img', visualization[img])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

**Part 2: Feature Matching**